<a href="https://colab.research.google.com/github/testa10/eccdum_assignments/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar el paquete eccd_datasets si es necesario
!pip install -q eccd_datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 MB 9.7 MB/s eta 0:00:00


In [ ]:
# Importar las bibliotecas necesarias
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
from eccd_datasets import load_images
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import io

# Cargar el dataset de eccd_datasets
dataset_df = load_images()  # DataFrame con columnas 'dataset', 'coarse_cat', 'finegrained_cat', 'image_data'

# Configuración básica
batch_size = 32
learning_rate = 0.001
num_epochs = 50
num_classes = len(dataset_df['coarse_cat'].unique())  # Contar las clases únicas en 'coarse_cat'

# Mapeo de etiquetas a índices numéricos
class_to_idx = {cls: idx for idx, cls in enumerate(dataset_df['coarse_cat'].unique())}
labels = dataset_df['coarse_cat'].map(class_to_idx).values  # Convertir etiquetas a índices numéricos

# Transformaciones de las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar a 224x224 para ResNet
    transforms.RandomHorizontalFlip(),  # Aumento de datos
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización
])

# Decodificar las imágenes y aplicar transformaciones
data_tensors = torch.stack([
    transform(Image.open(io.BytesIO(image_data))) for image_data in dataset_df['image_data']
])
label_tensors = torch.tensor(labels)  # Convertir etiquetas a tensores

# Crear el conjunto de datos y dividirlo en entrenamiento y validación
dataset = TensorDataset(data_tensors, label_tensors)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Configuración del dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo ResNet preentrenado y ajustar la última capa para clasificar frutas/verduras
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento del modelo
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Paso de entrenamiento
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    # Cálculo de la pérdida de entrenamiento promedio
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

print("Entrenamiento completado.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/50], Loss: 0.5326
Validation Accuracy: 82.73%
Epoch [2/50], Loss: 0.3335
Validation Accuracy: 87.35%
Epoch [3/50], Loss: 0.1862
Validation Accuracy: 94.40%
Epoch [4/50], Loss: 0.1354
Validation Accuracy: 88.81%
Epoch [5/50], Loss: 0.1838
Validation Accuracy: 94.40%
Epoch [6/50], Loss: 0.1849
Validation Accuracy: 83.94%
Epoch [7/50], Loss: 0.1411
Validation Accuracy: 97.08%
Epoch [8/50], Loss: 0.0831
Validation Accuracy: 75.18%
Epoch [9/50], Loss: 0.0762
Validation Accuracy: 92.46%
Epoch [10/50], Loss: 0.0663
Validation Accuracy: 92.46%
Epoch [11/50], Loss: 0.0977
Validation Accuracy: 88.08%
Epoch [12/50], Loss: 0.0810
Validation Accuracy: 99.27%
Epoch [13/50], Loss: 0.0824
Validation Accuracy: 91.48%
Epoch [14/50], Loss: 0.0205
Validation Accuracy: 96.59%
Epoch [15/50], Loss: 0.0429
Validation Accuracy: 95.62%
Epoch [16/50], Loss: 0.0594
Validation Accuracy: 92.46%
Epoch [17/50], Loss: 0.0605
Validation Accuracy: 93.67%
Epoch [18/50], Loss: 0.1145
Validation Accuracy: 94.40%
E

In [ ]:
# Definir precios de cada clase
class_prices = {
    "Apple": 74,
    "Pear": 89,
    "Satsumas": 44,
    "Melon": 109,
    "Orange": 35
}

# Convertir class_to_idx en idx_to_class para usar en el cálculo de costos
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Cálculo del costo en el conjunto de validación
model.eval()
total_cost = 0
total_correct = 0
total_samples = len(val_loader.dataset)

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        for i in range(len(labels)):
            true_class = idx_to_class[labels[i].item()]
            pred_class = idx_to_class[predicted[i].item()]

            # Obtener precios reales y predichos
            Yreal = class_prices[true_class]
            Ypred = class_prices[pred_class]

            # Calcular costo
            if Ypred > Yreal:
                cost = 5 * (Ypred - Yreal)
            elif Ypred < Yreal:
                cost = Yreal - Ypred
            else:
                cost = 0  # Caso en el que Ypred == Yreal

            total_cost += cost

            # Contar aciertos
            if Ypred == Yreal:
                total_correct += 1

# Calcular el costo promedio y la precisión
average_cost = total_cost / total_samples
accuracy = (total_correct / total_samples) * 100

print(f"Accuracy en el conjunto de validación: {accuracy:.2f}%")
print(f"Costo promedio por predicción: ${average_cost:.2f}")
print(f"Costo total en el conjunto de validación: ${total_cost:.2f}")

Accuracy en el conjunto de validación: 97.08%
Costo promedio por predicción: $1.88
Costo total en el conjunto de validación: $771.00
